<a href="https://colab.research.google.com/github/KingNook/QCES/blob/main/Project1_numerical_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Heat transport

In this project we will discretize and timestep two systems that describe heat transport. In the first part we will timestep a box model for the Earth's surface temperature with heat buffering provided by the deep ocean. In the second part, we will timestep the one-dimensional diffusion equation which can be used to describe heat moving through a medium (e.g. the ground or still air and water).

Through this project, we will learn how to efficiently timestep ordinary and partial differential equations and examine errors introduced by the numerical discretization. We will also have the chance to explore the timescales and physical processes involved in heat transport in the climate system.

## 1. Climate box model

In this part of the project we will timestep the following box model.

$$ C_s \partial_t T = F(t) - \gamma(T-\theta) $$

$$ C_d \partial_t \theta = \gamma (T-\theta) $$

Here, $T(t)$ represents the global mean surface temperature as a function of time and $\theta(t)$ is the temperature of the deep ocean, both expressed as the difference between the temperature and a pre-industrial equilibrium in units of $^\circ K$. The net surface heat flux, $F(t)$, drives departures from the pre-industrial temperatures. The heat flux between the surface and deep ocean is assumed to be proportional to the difference between the surface and deep ocean temperatures. The constants are the heat capacity of the surface of the Earth, $C_s$, the heat capacity of the deep ocean, $C_d$, and the heat flux coefficient, $\gamma$, all given in units of $W/^\circ K/m^2$. Note that $C_s$ and $C_d$ (as defined in Nicklas et al. 2025) are heat capacities per year; i.e. $1/C_s$ tells us how much the surface temperature would increase if a flux of 1 $W/m^2$ were applied for one year.

This model is inspired by a more complicated version of a two box climate model in the paper referenced below. (Interested students might consider adding some additional features from their model, but this is not required for this project).

*Nicklas, J.M., Fox-Kemper, B. and Lawrence, C., 2025. Efficient Estimation of Climate State and Its Uncertainty Using Kalman Filtering with Application to Policy Thresholds and Volcanism. Journal of Climate, 38(5), pp.1235-1270.*

Below, we start to build code to solve these equations. Execute the code cells below.  In some cases you will be asked to add code.

Student tasks are in [Section 1.1](#section1pt1), [Section 1.2](#section1pt2), and [Section 1.3](#section1pt3).

In [ ]:
# Before we start, import necessary libraries. Run this cell first.
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import time

Now, let's define values for the various constants. These values are taken from Nicklas et al., 2025.

In [ ]:
C_s = 17 # heat capacity of the Earth's surface (W/m^2/K)
C_d = 156 # heat capacity of the deep ocean (W/m^2/K)
gamma = 0.67 # heat exchange coefficient between surface and deep ocean (W/m^2/K)

Now, let's define a function for F(t), the net surface heat flux (in units of $W/m^2$) that will force our model. This function is here to represent the heat imbalance caused by antropogenic greenhouse gases in the atmosphere. When $F=0$, there is no net surface heat flux. The argument, $t$, is the time in years. We will let $t=0$ denote the start of our additional warming, and we set $F=1$ if $t<200$ years and $F=0$ if $t>200$ years. This corresponds to a very bad-case emissions scenario with an abrupt drop in emissions after 200 years.

In [ ]:
def F(t):
    return 1 if t < 200 else 0

Next, set the initial conditions. We will use arrays `T` and `theta`. `T[i]` represents the ith value in the array. We will use `T[0]` and `theta[0]` to hold the initial conditions. Subsequent values, `T[2]`, `T[3]`, etc. will be determined by timestepping the equations. We start by initializing the arrays using numpy and filling them with zeros and then set the initial values (this isn't necessary here since our values are zero anyway, but the code is here to that you can change the initial conditions easily)

In [ ]:
Nt = 500 # number of time steps (including the starting point)
T = np.zeros(Nt) # surface temperature array
theta = np.zeros(Nt) # deep ocean temperature array
T[0] = 0 # initial surface temperature anomaly (ºK)
theta[0] = 0 # initial deep ocean temperature anomaly (ºK)

Now, create an array with the time (in years) corresponding to each time index. Also, calculate the timestep, `dt`.

In [ ]:
t = np.linspace(0, 500, Nt) # time in years.  This starts at 0 and ends at 500, with Nt total points
dt = t[1] - t[0] # timestep in years

Finally, we are ready to create a for loop to timestep the equations listed at the top of the section.

### 1.1 Student task:
<a id='section1pt1'></a>
Fill in the interior of the for loop to timestep the equations from the start of Section 1 using the Explicit Euler method.

In [ ]:
def flux(i, T=T, theta=theta):
    return gamma * (T[i] - theta[i])

for i in range(0, Nt-1):
    T[i+1] = T[i] + dt * (F(t[i])  - flux(i))/C_s
    theta[i+1] = theta[i] + dt * flux(i)/C_d

After you are finished, run the cell below to plot the results

In [ ]:
plt.plot(t,T,'r-')
plt.plot(t,theta,'b-')
plt.legend(['Surface temperature','Deep ocean temperature'])
plt.xlabel('Time (years)')
plt.ylabel('Temperature anomaly (ºK)')
plt.show()

### 1.2 Student task
<a id='section1pt2'></a>
Discuss the rates of change the temperature of the Earth's surface and the deep ocean at the start of the model run.  Can you explain the response of $T$ and $\theta$ to the forcing by defining characteristic timescales from the box model equations?

**Description**

Rate of change of temperature at the surface is initially at its maximum; gradient deecreases over time as the rate of change of deep ocean temperature increases, until both reach the an equilibrium, at which point they increase at the same rate. Once the forcing stops (ie at t=200, when F(t) becomes 0), the surface temperature decreases rapidly, while the rate of increase of the deep sea temperature decreases until they reach equilibrium

**Response to Forcing**

The forcing occurs in the surface ocean, so that is where the temperature change initially occurs. There is then a 'plateau-ing' effect from the flux term, $\gamma (T - \theta)$, which also corresopnds to the increase in deep ocean temperature. The flux is proportional to the difference between the surface / deep ocean temperatures, and if we take a further derivative, we see that the flux becomes constant when their rates of change are equal (ie when their gradients are equal), thus explaining why they tend to parallel lines. The reverse of this description basically describes the behaviour once forcing stops

**Time Scales**

There is a warmup period and a cool-down period; from the eyeball test, it looks like both are around t=100. Through exact calculation, we find that, if we define the flux as 

$$
Q = \gamma (T-\theta)
$$

then the derivative is 

$$
\begin{aligned}
\partial_t Q &= \frac{\gamma}{C_s} \partial_t F - \gamma (\frac{1}{C_s} + \frac{1}{C_d}) Q \\
&= - AQ
\end{aligned}
$$

since $F$ is independent of time, and the pre-factor in front of Q is constant. Thus we see that $Q\propto e^{-At}$ and thus that the time scale of the curved part of the graph (ie the interesting stuff) is $\tau = 1/A \approx 23$?? (which seems wrong idk -- it looks like it takes around 100-120s to settle into a steady state)

In [ ]:
1/(gamma * (1/C_s + 1/C_d))

### 1.3 Student task
<a id='section1pt3'></a>
Modify the forcing function, `F(t)` above with 2-3 scenarios of your choice. Plot the temperature response for each scenario and briefly (2-3 sentences each) describe the temperature responses in each scenario. You will need to replicate some of the code blocks above so that you can produce a new set of plots below.

In [ ]:
from numpy import sin, pi

## Options for F

def F_1(t):
    ## Non-constant forcing
    return t

def F_2(t):
    ## Warming then constant then cooling
    return 1 if t < 100 else 0 if t < 200 else -1

def F_3(t):
    ## Stepped forcing
    return t//150

def F_4(t):
    ## Oscillating Forcing
    return sin(2 * pi * t/50)

In [ ]:
## Options for F

def F(t):
    return F_4(t)

## Initialise
Nt = 500
T = np.zeros(Nt) # surface temperature array
theta = np.zeros(Nt) # deep ocean temperature array
T[0] = 0 # initial surface temperature anomaly (ºK)
theta[0] = 0 # initial deep ocean temperature anomaly (ºK)

t = np.linspace(0, 500, Nt) # time in years.  This starts at 0 and ends at 500, with Nt total points
dt = t[1] - t[0] # timestep in years

def flux(i, T=T, theta=theta):
    return gamma * (T[i] - theta[i])

for i in range(0, Nt-1):
    T[i+1] = T[i] + dt * (F(t[i])  - flux(i))/C_s
    theta[i+1] = theta[i] + dt * flux(i)/C_d

# Add code to make plots here
plt.plot(t,T,'r-')
plt.plot(t,theta,'b-')
plt.legend(['Surface temperature','Deep ocean temperature'])
plt.xlabel('Time (years)')
plt.ylabel('Temperature anomaly (ºK)')
plt.show()

**Thoughts / Discussion**

$F_1$: (Non-constant forcing) We see that in the case of forcing that is linear in time, the surface temperature and ocean do not actually reach a steady state

$F_2$: (Warming then cooling) About as expected; when surface is cooler than deep ocean, the flux reverses direction

$F_3$: (Stepped increases) After each increase in forcing, there is an adjustment period before the system settles down into a steady state again

$F_4$: (Oscillatory forcing) We see that the surface having oscillatory forcing results in oscillatory behaviour in the deep ocean as well, with (ostensibly) the same period, but out of phase by 1/4 cycle (or equivalently $\pi/2$ radians)

## 2. Diffusion equation

In [ ]:
# Before we start, import necessary libraries. Run this cell first.
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import time

In this part of the project we will solve the 1-D diffusion equation:

$$ \partial_t c = \kappa \partial_x^2 c $$

We will use second order finite differences to approximate the $x$-derivatives, and we will implement and compare two timestepping methods.

Student tasks are in [Section 2.1](#section2pt1), [Section 2.2](#section2pt2), and [Section 2.3](#section2pt3).

First, let's discretize the second order derivative using finite differences. The function below builds a coefficient matrix, $M$, so that $Mc$ is a discretized version of $\partial^2_x c$, where in our discretized version $c$ is a column vector.

In [ ]:
def construct_M(x):
    # Build a coefficient matrix, such that M*c is the discrete version of d^2/dx^2(c)
    dx = x[1] - x[0]  # Infer the uniform grid spacing
    M = (np.diag(np.ones(Nx-1), -1) - 2*np.diag(np.ones(Nx), 0) + np.diag(np.ones(Nx-1), 1)) / dx**2
    M = sp.csr_matrix(M) # Turn M into a sparse matrix to save memory and improve speed
    return M

Now, define two functions to timestep the diffusion equation. The code below timesteps the diffusion equation using the expicit Euler method. The function takes in a vector of discrete positions, `x`, discrete times, `t`, and the diffusivity, `kappa`. The vectors `x` and `t` dictate the discretization of the problem in space and time, and the grid spacing must be constant in space and time. The boundary conditions are $\partial_x c=0$ at the starting and ending points in `x`.

In [ ]:
def EE(x, t, kappa):
# This function timesteps the diffusion equation with homogeneous Neumann boundary conditions
# using the explicit Euler and second order finite difference methods

    start_time = time.time()  # Start a timer

    # Infer the timestep from the time array
    dt = t[1] - t[0]

    M = construct_M(x)  # Construct the coefficient matrix for the second derivative

    # Set the number of gridpoints in x and t
    Nx = len(x); Nt = len(t)

    # Create an empty sparse array to store the solution
    c = np.zeros((Nx, Nt))

    c[:, 0] = set_ics(x, kappa, 10)  # Set the initial condition

    for k in range(1, Nt):  # loop over the number of timesteps
      c[:,k] = c[:,k-1] + dt * kappa * M @ c[:,k-1]
      # Enforce the boundary conditions (no flux)
      c[0, k] = c[1, k]
      c[-1, k] = c[-2, k]

    end_time = time.time()  # Print the elapsed time

    print(f"Explicit Euler method took {end_time - start_time:.2f} seconds")

    return c

Similarly, the function below timesteps the diffusion equation with the Crank-Nicolson method

In [ ]:
def CN(x, t, kappa):
# This function timesteps the diffusion equation with homogeneous Neumann boundary conditions
# using the explicit Euler and second order finite difference methods
    start_time = time.time()  # Start a timer

    # Infer the timestep from the time array
    dt = t[1] - t[0]

    M = construct_M(x)  # Construct the coefficient matrix for the second derivative

    # Set the number of gridpoints in x and t
    Nx = len(x); Nt = len(t)

    A = sp.eye(Nx) - 0.5 * dt * kappa * M  # This matrix appears on the LHS in the C-N scheme
    A_inv = spla.inv(A) # Calculate the inverse of A (this only needs to be done once)

    # Create an empty sparse array to store the solution
    c = np.zeros((Nx, Nt))

    c[:, 0] = set_ics(x, kappa, 10)  # Set the initial condition

    for k in range(1, Nt): # loop over the number of timesteps
        c[:, k] = A_inv @ (c[:, k-1] + 0.5 * dt * kappa * (M @ c[:, k-1]))
        # Enforce the boundary conditions
        c[0, k] = c[1, k]
        c[-1, k] = c[-2, k]

    end_time = time.time()  # Print the elapsed time

    print(f"Crank-Nicolson method took {end_time - start_time:.2f} seconds")

    return c

Next, create a function to set the initial conditions. The exact solution to the unbounded diffusion equation with a delta function initial condition is:

$$\left(\frac{1}{4\pi\kappa t}\right)^{1/2}e^{-x^2/(4\kappa t)}$$

The function below sets the initial condition by evaluating the exact solution at a given time. Note that it is important that we don't evaluate the exact solution at `t_ic=0` since this will return a delta function which will be undefined on our finite grid. The value of `t_ic` is set within the `EE` and `CN` functions.

In [ ]:
# Function to set the initial condition from the exact solution at time t_start
def set_ics(x, kappa, t_ic):
    c_ic = (4*np.pi*kappa*t_ic)**(-0.5)*np.exp(-x**2 / (4*kappa*t_ic))
    # Enforce the boundary conditions
    c_ic[0] = c_ic[1]
    c_ic[-1] = c_ic[-2]
    return c_ic


Finally, define some parameters and the x and t vectors:

In [ ]:
# Defining parameters
kappa = 0.001  # diffusivity

# Timestepping parameters
t_start = 0  # start time (note that this is not the same time used to evaluate the exact solution to set the ICs)
t_end = 10  # end time
Nt = 1000  # number of timesteps

# Size of the physical domain
x_start = -1
x_end = 1
Nx = 100

t = np.linspace(t_start, t_end, Nt)  # Create a vector of times
x = np.linspace(x_start, x_end, Nx)  # Create a vector for x

Run each method once:

In [ ]:
# Run the simulations
c_EE = EE(x, t, kappa) # explicit Euler
c_CN = CN(x, t, kappa) # Crank-Nicolson

and finally, plot the results side-by-side

In [ ]:
# Plot the results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.pcolormesh(x, t, c_EE.T, shading='auto')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Explicit Euler')

plt.subplot(1, 2, 2)
plt.pcolormesh(x, t, c_CN.T, shading='auto')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Crank-Nicolson')

plt.tight_layout()
plt.show()

## 2. Student investigation

### 2.1 Stability
<a id='section2pt1'></a>
The explicit Euler and Crank-Nicolson methods have different stability properties. To explore this, try increasing the diffusivity, $\kappa$, and see how each method responds. Discuss your findings below (~2-3 sentences).

In [ ]:
### Original Code

## General Parameters
# Timestepping parameters
t_start = 0  # start time (note that this is not the same time used to evaluate the exact solution to set the ICs)
t_end = 10  # end time
Nt = 1000  # number of timesteps

# Size of the physical domain
x_start = -1
x_end = 1
Nx = 100

t = np.linspace(t_start, t_end, Nt)  # Create a vector of times
x = np.linspace(x_start, x_end, Nx)  # Create a vector for x

# kappa_list = np.linspace(0.01, 0.03, 20)
kappa_list = np.linspace(15000, 25000, 11)
fig, axes = plt.subplots(len(kappa_list), 2, figsize = (14, 6*len(kappa_list)))

def plot_stability(c_EE, c_CN, x=x, t=t, axis=axes, kappa=0):

    axis[0].pcolormesh(x, t, c_EE.T, shading='auto')
    axis[0].set_xlabel('x')
    axis[0].set_ylabel('t')
    axis[0].set_title(f'Explicit Euler // kappa={kappa:.02f}')
    #plt.colorbar(mappable = ref, ax = axis[0], extend='both')

    ref = axis[1].pcolormesh(x, t, c_CN.T, shading='auto')
    axis[1].set_xlabel('x')
    axis[1].set_ylabel('t')
    axis[1].set_title(f'Crank-Nicolson // kappa={kappa:.02f}')

    plt.colorbar(mappable = ref, ax = axis[1], extend='both')



def run_sim(kappa = 0.001):

    # Run the simulations
    c_EE = EE(x, t, kappa) # explicit Euler
    c_CN = CN(x, t, kappa) # Crank-Nicolson

    return c_EE, c_CN


def multi_run_sim(kvals = kappa_list, axes = axes):

    for i, kappa in enumerate(kvals):
        c_EE, c_CN = run_sim(kappa)
        plot_stability(c_EE, c_CN, axis = axes[i], kappa=kappa)

multi_run_sim()

plt.tight_layout()
plt.show()

#### Setup (Run before either sim)

In [ ]:
## General Parameters
# Timestepping parameters
t_start = 0  # start time (note that this is not the same time used to evaluate the exact solution to set the ICs)
t_end = 10  # end time
Nt = 1000  # number of timesteps

# Size of the physical domain
x_start = -1
x_end = 1
Nx = 100

t = np.linspace(t_start, t_end, Nt)  # Create a vector of times
x = np.linspace(x_start, x_end, Nx)  # Create a vector for x

#### Investigation of Explicit Euler

In [ ]:
n_vals = 10

kappa_list = np.linspace(0.015, 0.025, n_vals)
fig, axes = plt.subplots(n_vals//2, 2, figsize = (14, 3*n_vals))

for k_idx, kappa in enumerate(kappa_list):

    c_EE = EE(x, t, kappa)

    row, col = divmod(k_idx, 2)
    ax = axes[row][col]
    
    ref = ax.pcolormesh(x, t, c_EE.T, shading='auto')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_title(f'Explicit Euler // kappa={kappa:.04f}')

    plt.colorbar(mappable = ref, ax = ax, extend='both')

plt.tight_layout()
plt.show()

In [ ]:
n_vals = 10

kappa_list = np.linspace(1, 100, n_vals)
fig, axes = plt.subplots(n_vals//2, 2, figsize = (14, 3*n_vals))

for k_idx, kappa in enumerate(kappa_list):

    c_CN = CN(x, t, kappa)

    row, col = divmod(k_idx, 2)
    ax = axes[row][col]
    
    ref = ax.pcolormesh(x, t, c_CN.T, shading='auto')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.set_title(f'Explicit Euler // kappa={kappa:.04f}')

    plt.colorbar(mappable = ref, ax = ax, extend='both')

plt.tight_layout()
plt.show()

**Stability Observations**

From the theory, we can calculate that the Explicit Euler scheme should lose stability at $\kappa=0.02$ -- we see this is corroborated by experimental results (as can be seen in the above code block). Note the color bar along the side being in the order of $10^{\text{large number}}$ suggesting instability.

Crank-Nicolson does not seem to lose stability for any $\kappa$; we see at extreme $\kappa$ (in the order of $10^4$ which is probably unphysical), we get some slightly odd-looking behaviour with the alternating stripes, but looking at the absolute values, we see this simply alternates between $\pm 1e-4$ ie the simulation is effectively at 0 the entire time, and that stability is still not lost. I suspect these are driven by boundary conditions?

### 2.2 Accuracy
<a id='section2pt2'></a>
Next, we will examine the accuracy of the numerical methods used above. In order to do this, it is useful to compare our numerical solutions with the exact solution to the diffusion equation. Below, write code to plot the exact solution and the numerical solution using explicit Euler and Crank-Nicolson at the final time ($t=10$). Note that you can use the exact solution to the diffusion equation in an unbounded domain in $x$ and keep in mind that the exact solution and numerical solutions will differ for later times when boundary effects are felt.

In [ ]:
# Add code to plot exact and numerical solutions
fig, ax = plt.subplots()
t_ic = 10
# exact_soln = lambda x, t, kappa: (1 / np.sqrt(4 * np.pi * kappa * t)) * np.exp(- x**2 / (4 * kappa * t)) if t != 0 else (1 / np.sqrt(4 * np.pi * kappa * t_ic)) * np.exp(- x**2 / (4 * kappa * t_ic))
exact_soln = lambda x, t, kappa: (1 / np.sqrt(4 * np.pi * kappa * (t + t_ic))) * np.exp(- x**2 / (4 * kappa * (t + t_ic)))

kappa = 0.015
t_val = 0

# Run the simulations
final_EE = EE(x, t, kappa) # explicit Euler
final_CN = CN(x, t, kappa) # Crank-Nicolson
final_exact = exact_soln(x, t_val, kappa)

ax.plot(x, final_EE[:, 0], label = 'Explicit Euler')
ax.plot(x, final_CN[:, 0], label = 'Crank Nicolson')
ax.plot(x, final_exact, label = 'Exact Soln')

ax.set_xlim(-1, 1)
ax.set_ylim(0, 1)

ax.set_title(f'Comparison to Exact Solution at time = {t_val}')

ax.legend()
plt.show()

The difference between the exact solution and the numerical approximation can be quantified by various measures of error. The *local error* is the difference between the exact solution and the numerical approximation after one timestep. The *global error* is the difference between the solutions after a fixed amount of time (which includes the accumulation of multiple steps of local error). A method is first order accurate if the global error is linearly proportional to the timestep. A method is second order accurate if the global error varies quadratically with the timestep. Below, calculate and plot the global error for the Explicit Euler and Crank-Nicolson methods and use your results to infer the order of accuracy of each scheme.

PLOT GLOBAL ERROR FOR EACH SCHEME (1-2 figures) AND DISCUSS ORDER OF ACCURACY (2-3 sentences)

In [ ]:
# Add code to plot error here
def mse(pred, actual):
    return ((actual - pred)**2).mean()

EE_global_error = []
CN_global_error = []

for t_idx, t_val in enumerate(t):

    true_soln = exact_soln(x, t_val, kappa)

    EE_error = mse(final_EE[:, t_idx], true_soln)
    CN_error = mse(final_CN[:, t_idx], true_soln)

    EE_global_error.append(EE_error)
    CN_global_error.append(CN_error)

    print(f'[t={t_val:.02f}]Errors: EE {EE_error:.02f}, CN {CN_error:.02f}')

In [ ]:
## Plotting global error

fig, ax = plt.subplots()

ax.plot(t, EE_global_error, color = 'blue', label = 'Explicit Euler')
ax.plot(t, CN_global_error, color = 'orange', label = 'Crank-Nicolson')

ax.legend()

ax.set_xlim(0, 10)
ax.set_ylim(0)
ax.set_title(f'Global Errors at $\\kappa = {kappa}$')

Finally, select a given error tolerance and explore whether explicit Euler or Crank-Nicoslon is more efficient (faster) at generating solutions with error less than your error tolerance. Discuss your conclusion and add supporting figures if necessary.

**Better Errors**

From the plots, it seems that Crank-Nicolson is just better than Explicit Euler over all values of $\kappa$ for this particular question. As for time, that still is yet to be coded by me

### 2.3 Boundary conditions for Crank-Nicolson scheme
<a id='section2pt3'></a>

The application of the boundary conditions for the Crank-Nicolson method isn't quite right in the script above (but the boundary conditions aren't very important in this example). Account for the boundary conditions by modifying the coefficient matrix. This should ensure that the boundary conditions are applied to the implicit and explicit terms involved in the Crank-Nicolson update. Plot your results below and verify that the boundary conditions are satisfied.  How would you handle fixed value (Dirichlet) and fixed gradient (Neumann) boundary conditions?

In [ ]:
ADD CODE AND PLOTS HERE

ADD DISCUSSION OF DIRICHLET AND NEUMANN BOUNDARY CONDITIONS HERE (NO CODE NEEDED)